In [1]:
import numpy as np
import time
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [2]:
timesteps = 100
samplingtime = 0.2
maxsetpoint = 1.0
timepoints = []
outputs = []
setpoints = []
errors = []

timepoints = np.zeros(timesteps)
outputs = np.zeros(timesteps)
setpoints = np.zeros(timesteps)
errors = np.zeros(timesteps)

In [3]:
class PID:
    def __init__(self, P, I, D):
        self.kp = P
        self.ki = I
        self.kd = D

    def initialize(self):
        # Initialize
        global timepoints 
        timepoints.fill(np.nan)
        global outputs 
        outputs.fill(np.nan)
        global setpoints 
        setpoints.fill(np.nan)
        global errors 
        errors.fill(np.nan)
        self.processval = 0.0
        self.controlvar = 0.0
        self.setpoint = 0.0
        self.error = 0.0
        self.prop_term = 0.0
        self.inte_term = 0.0
        self.deri_term = 0.0
        self.lasterr = 0.0
        self.currenttime = time.time()
        self.lasttime = self.currenttime
    
    def controlLoop(self):
        # PID control logic
        self.error = self.setpoint - self.processval
        self.currenttime = time.time()
        dt = self.currenttime - self.lasttime
        if dt > samplingtime:
            self.prop_term = self.kp * self.error
            inte_term = self.inte_term + self.ki * self.error * dt
            self.deri_term = self.kd * (self.error - self.lasterr) / dt
            self.controlvar = self.prop_term + self.inte_term + self.deri_term
            self.lasterr = self.error
            self.lasttime = self.currenttime
        time.sleep(samplingtime)

In [4]:
# Setup plots
p1 = figure(plot_width=400, plot_height=300, 
            x_range=[0, timesteps * samplingtime], title='PID controller input and output')
p2 = figure(plot_width=400, plot_height=300, 
            x_range=[0, timesteps * samplingtime], title='PID controller error')
r1 = p1.line(timepoints, setpoints, line_color='cornflowerblue', legend='input',
            line_width=2)
r2 = p1.line(timepoints, outputs, line_color='indianred', legend='output',
            line_width=2)
r3 = p2.line(timepoints, errors, line_color='indigo', legend='error',
            line_width=2)
p1.legend.location = 'top_left'  
p2.legend.location = 'top_left'  
p1.toolbar.logo = None
p1.toolbar_location = None
p2.toolbar.logo = None
p2.toolbar_location = None

In [5]:
def runPID(InputFunction='step', Kp=0.75, Ki=0.5, Kd=0.05):
    pid = PID(Kp, Ki, Kd)
    pid.initialize()
    for nn in range(1, timesteps, 1):
        # Inputs
        if InputFunction == 'step': 
            # step function
            if nn > 30:
                pid.setpoint = maxsetpoint
        elif InputFunction == 'square':
            # square wave
            if nn > 30:
                pid.setpoint = maxsetpoint * np.sign(np.sin(4 * np.pi * (nn - 30) / timesteps))
        elif InputFunction == 'sine':
            # sine wave
            if nn > 30:
                pid.setpoint = maxsetpoint * np.sin(4 * np.pi * (nn - 30) / timesteps)
        # Outputs 
        if nn > 30:
            pid.processval = pid.processval + (pid.controlvar - (1.0 / nn))
        pid.controlLoop()
        global outputs
        outputs[nn] = pid.processval
        global timepoints
        timepoints[nn] = nn * samplingtime
        global setpoints
        setpoints[nn] = pid.setpoint
        global errors
        errors[nn] = pid.error
        # Update plots
        r1.data_source.data['x'] = timepoints
        r1.data_source.data['y'] = setpoints
        r2.data_source.data['x'] = timepoints
        r2.data_source.data['y'] = outputs
        r3.data_source.data['x'] = timepoints
        r3.data_source.data['y'] = errors
        push_notebook(handle=target)

In [6]:
target = show(row(p1, p2), notebook_handle=True)
interact(runPID, InputFunction=['sine', 'square', 'step'], 
         Kp=(0, 1.5, 0.25), Ki=(0, 1.0, 0.2), Kd=(0, 0.1, 0.05))

interactive(children=(Dropdown(description='InputFunction', index=2, options=('sine', 'square', 'step'), value…

<function __main__.runPID(InputFunction='step', Kp=0.75, Ki=0.5, Kd=0.05)>